# Example Notebook to Download Alpha Earth Embeddings at Scale

In [ ]:
# !pip install geedim

## Import Libraries

In [ ]:
import ee
import os
import geedim as gd
import geopandas as gpd
import rioxarray as rxr

from glob import glob
from pathlib import Path
from shapely.geometry import box, mapping

## Setup Credentials

In [ ]:
gee_account = 'replace_with_your_service_account' # or just replace the auth with Authenticate
gee_key = 'replace_with_your_service_key.json'

In [ ]:
# get credentials
credentials = ee.ServiceAccountCredentials(gee_account, gee_key)
ee.Initialize(credentials)  # gd initialize does not take service account
print("Initialized")

## Open Geospatial Data

In [ ]:
aoi_filename = 'boreal_tiles_v004_model_ready.gpkg'

In [ ]:
aoi_gdf = gpd.read_file(aoi_filename).to_crs(4326)
aoi_gdf_crs = aoi_gdf.crs.to_epsg()
aoi_gdf

## Extract Embeddings from a Single Cell

In [ ]:
geom = aoi_gdf.geometry.iloc[0]
ee_geom = ee.Geometry(mapping(geom))
year = 2023 # year to download
scale = 30 # resolution
nodata = -9999

In [ ]:
output_filename = f'alpha_earth_{year}_{scale}m.tif'

In [ ]:
aoi_gdf_crs

### No Filtering

In [ ]:
# get collection
gee_image_collection = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL') \
    .filterBounds(ee_geom) \
    .filterDate(f'{year}-01-01', f'{year+1}-01-01')

# clip the imagery again
gee_image = gee_image_collection.toBands().clip(ee_geom)

im = gd.mask.MaskedImage(gee_image)
im.download(output_filename, region=ee_geom, crs=f'EPSG:{aoi_gdf_crs}', scale=scale)

### Filtering First Five Bands

In [ ]:
# get collection
gee_image_collection = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL') \
    .filterBounds(ee_geom) \
    .filterDate(f'{year}-01-01', f'{year+1}-01-01')

# clip the imagery again
gee_image = gee_image_collection.toBands().clip(ee_geom)

# get the first 5 band names
band_names = gee_image.bandNames().slice(0, 5)

# select only those bands
gee_image = gee_image.select(band_names)
print('Downloading:', f'{len(gee_image.bandNames().getInfo())} bands')

im = gd.mask.MaskedImage(gee_image)
im.download(output_filename, region=ee_geom, crs=f'EPSG:{aoi_gdf_crs}', scale=scale)

In [ ]:
# you can also download it as numpy arrays, xarray, etc.; for more information reference
# https://geedim.readthedocs.io/en/stable/getting_started/api.html#numpy